# Practica 5. _Analisis de un sistema de recomendación basado en modelos_
- **Hecho por:** Andre Pires
- **Objetivo:** Implementar un filtro colaborativo basado en factorización matricial de bajo rango (se puede utilizar el algoritmo SVD de Surprise). Además, realizarán actividades complementarias para profundizar en la interpretación, evaluación y aplicación de los modelos de recomendación

## Introduccion
En esta práctica se implementa un sistema de recomendación basado en filtrado colaborativo por modelos. La idea es que, a partir de los ratings históricos, entender un modelo paramétrico que explique las valoraciones y permita predecir las que faltan para recomendar ítems no vistos.

Trabajaremos con la formulación clásica de factorización matricial de bajo rango con sesgos donde cada usuario $u$ y cada item $i$ tienen un vector de factores latentes y, además, se modelan sesgos. La prediccion nos queda de la siguiente forma:

$r_{ui} = \mu + b_u + b_i + p_u^{\top} q_i$

Estos parámetros se aprenden por descenso del gradiente con regularización para evitar sobreajuste. Esta aproximación se popularizó en el contexto del Netflix Prize y está implementada en la librería Surprise que se usa.

#### Probelmas previos
En Windows tuve conflicto Surprise y NumPy 2.3.4. Primero, para solucionarlo, cree un entorno con la version de python que correspondiense y encajase a la prefeccion para el uso de surprise. Despues, desinstale y fije numpy en al version 1.26.4 y reinstale scikit-surprise en la version 1.1.4, ya que estas eran compatibles sin crearme ningun proeblema. Esta parte me llevo alguna horita y entendi que los wheels de Surprise aun no soportan NumPy 2.x y que por tanto no podian existir un version de Numpy 2.x

## Practica
---

### Busqueda de archivo y rutas
Primero de todo vamos a buscar los ficheros y establecer las rutas para su uso 

In [1]:
from pathlib import Path
ruta_ratings  = Path("dataset/ml-latest-small/ratings.csv")
ruta_peliculas = Path("dataset/ml-latest-small/movies.csv") 

### Carga
Tras tener los ficheros, deberemos cargarlos para poder trabajar con ellos. en este punto aprovecharemos a validar para mas adelante, cuando estemos trabajando con ellos, no nos den errores noe esperados

In [2]:
import pandas as pd

# Cargar datos en DataFrames
ratings = pd.read_csv(ruta_ratings)      
peliculas = pd.read_csv(ruta_peliculas)  

# Comprobamos que las columnas son correctas
columnas_necesarias_ratings = {"userId", "movieId", "rating"}
columnas_necesarias_peliculas = {"movieId", "title"}
if not columnas_necesarias_ratings.issubset(set(ratings.columns)):
    raise ValueError("El archivo no tiene las columnas esperadas")
if not columnas_necesarias_peliculas.issubset(set(peliculas.columns)):
    raise ValueError("El archivo no tiene las columnas esperadas")

# Forzamos a tipos consistentes
ratings["userId"] = ratings["userId"].astype(str)
ratings["movieId"] = ratings["movieId"].astype(str)
peliculas["movieId"] = peliculas["movieId"].astype(str)

display(ratings.head(3))
display(peliculas.head(3))

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


### Preparacion de Datos para Surprise
El siguiente paso, crucial para trabajar con la librería Surprise, es adaptar nuestro DataFrame de pandas a la estructura de datos que espera esta librería, conocida como un Dataset. Además, necesitamos crear un Reader que especifique el rango de las valoraciones en nuestro conjunto de datos.
#### Rangos
El dataset MovieLens de 100k utiliza un rango de calificaciones de 0.5 a 5.0 estrellas. El objeto Reader de Surprise necesitara esta información por lo que la guardamos
#### Carga
Usaremos el método load_from_df ya que es un metodo que porporciona un uso bastante sencillo para machine learning y procesamiento de lenguaje natural

In [3]:
from surprise import Dataset, Reader

rango_minimo_rating = 0.5
rango_maximo_rating = 5.0

lector_datos = Reader(rating_scale=(rango_minimo_rating, rango_maximo_rating))
datos_surprise = Dataset.load_from_df(ratings[["userId", "movieId", "rating"]], lector_datos)

### Partición, entrenamiento SVD y métricas
En este punto implementaremos el filtro colaborativo basado en factorización matricial utilizando el algoritmo SVD de la librería Surprise, que incluye el modelado de sesgos ($b_u$ y $b_i$).

El modelo se entrenará utilizando el conjunto de datos completo. En esta practica no se hace una división en train/test por ahora.

Para este paso definiremos el algoritmo y lo entrenaremos con una muestra del conjunto de datos completo, ya que Surprise opera sobre un set de entrenamiento.

In [4]:
from surprise import SVD, accuracy
from surprise.model_selection import train_test_split

# Partimos en entrenamiento/test
conjunto_entrenamiento, conjunto_prueba = train_test_split(
    datos_surprise,
    test_size=0.2,
    random_state=42
)

# Definimos el modelo SVD con sesgos
modelo = SVD(
    n_factors=50,   # dimensión latente (ajustable)
    n_epochs=20,    # pasadas de SGD
    lr_all=0.005,   # learning rate
    reg_all=0.02,   # regularización
    random_state=42
)

# Entrenamos
modelo.fit(conjunto_entrenamiento)

# Evaluamos
predicciones = modelo.test(conjunto_prueba)
rmse = accuracy.rmse(predicciones, verbose=False)
mae  = accuracy.mae(predicciones,  verbose=False)

print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")

RMSE: 0.8775
MAE : 0.6742


### Añadir mi usuario
Siguiendo el guion dado, deberemos de:

_"Añadir uno o varios usuarios que representen a los miembros del equipo de prácticas (con ratings a un subconjunto de las películas del dataset), ajustar el filtro y mostrar las 10 mejores recomendaciones que proporciona a cada usuario añadido"_. 

Por tanto, deberemos crear un usuario y reentrenar el modelo con sus escasos datos para despues generar una lista de recomendaciones Top-10

#### Definimos usario
Para definir un usuario con sus valoraciones podriamos hacerlo de varaias formas. En nuestro caso lo haremos con la creacion directa de un dataFrame. Esot lo hacemos ya que nos ayudara con la simulacion de datos y la preparacion para el reentrenamiento del modelo SVD.

In [5]:
Idusario = 611

valoraciones_usuario = pd.DataFrame([
    [Idusario, "144478",   5.0],
    [Idusario, "54908", 4.5],
    [Idusario, "480", 2.0],
    [Idusario, "2640", 5.0],
    [Idusario, "95165",  4.0],
], columns=["userId","movieId","rating"])


#### Rentreno de nuestro modelo con el usuario nuevo 
Tras tener un usuario nuevo ya creado on sus valoraciones, deberemos de reentrenarlo teniendo en cuenta esas nuevas valoracines

In [6]:
ratings_combinados = pd.concat([ratings, valoraciones_usuario], ignore_index=True)

# Recargamos el dataset completo para Surprise con los nuevos ratings
datos_surprise_reentrenamiento = Dataset.load_from_df(
    ratings_combinados[['userId', 'movieId', 'rating']],
    lector_datos
)

conjunto_entrenamiento_nuevo = datos_surprise_reentrenamiento.build_full_trainset()

modelo_reentrenado = SVD(
    n_factors=50,  
    n_epochs=20,    
    lr_all=0.005,   
    reg_all=0.02,   
    random_state=42
)

# Entrenamos
modelo_reentrenado.fit(conjunto_entrenamiento_nuevo)

#### Top 10 
Para recomendar, identificamos las películas no vistas por el nuevo usuario y usamos el modelo reentrenado para predecir su rating

In [7]:
peliculas_vistas_andre = valoraciones_usuario['movieId'].tolist()

peliculas_no_vistas = []
for id_pelicula in peliculas['movieId'].unique():
    if id_pelicula not in peliculas_vistas_andre:
        peliculas_no_vistas.append(id_pelicula)

predicciones = []
for id_pelicula in peliculas_no_vistas:
    # Usamos el modelo reentrenado 
    pred = modelo_reentrenado.predict(Idusario, id_pelicula)
    # r_ui = mu + b_u + b_i + p_u^T * q_i
    predicciones.append((id_pelicula, pred.est))

# Las ordenamos
predicciones.sort(key=lambda x: -x[1])
top10 = predicciones[:10]

df_top10 = pd.DataFrame(top10, columns=["movieId","prediccion"]).merge(peliculas, on="movieId")
print("Top 10 con estimaciones del usuario nuevo")
display(df_top10)

Top 10 con estimaciones del usuario nuevo


,movieId,prediccion,title,genres
0,318,4.635178,"Shawshank Redemption, The (1994)",Crime|Drama
1,1197,4.602836,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
2,1204,4.600398,Lawrence of Arabia (1962),Adventure|Drama|War
3,898,4.551514,"Philadelphia Story, The (1940)",Comedy|Drama|Romance
4,27773,4.547040,Old Boy (2003),Mystery|Thriller
5,1104,4.537197,"Streetcar Named Desire, A (1951)",Drama
6,2959,4.528547,Fight Club (1999),Action|Crime|Drama|Thriller
7,246,4.509483,Hoop Dreams (1994),Documentary
8,1262,4.503211,"Great Escape, The (1963)",Action|Adventure|Drama|War
9,56782,4.498510,There Will Be Blood (2007),Drama|Western


### Sesgos

#### Sesgo peliculas
La componente del sesgo de item, $b_i$, nos permite identificar que peliculas reciben valoraciones consistentemente más altas o más bajas que la media global de ratings ($\mu$), independientemente de las preferencias latentes del usuario. Dependiendo del tipo de sesgo encontramos lo siguiente:
- Sesgo positivo: inidica que las peliculas son mas atractivas y, por tanto, tienen unas puntuaciones mayores por la mayoria de usuario
- Sesgo negativo: Señala peliculas que, en promedio, tienen un resultando en puntuaciones consistentemente bajas

En resumen, este punto nos ayuda a comprender de una vistazo rapido las peliculas con mas popularidad y menos popularidad

In [8]:
# Usamos get_raw_item_biases() para obtener la lista de pares (ID_interno, sesgo_i)
sesgos_bi = modelo_reentrenado.bi

# Sesgos peliculas
lista_sesgos_peliculas = []
for id_interno_item in conjunto_entrenamiento_nuevo.all_items():
    # Obtener el sesgo usando el índice (ID interno)
    sesgo_valor = sesgos_bi[id_interno_item]
    
    # Mapear el ID interno al ID de película original (string)
    id_pelicula_original = conjunto_entrenamiento_nuevo.to_raw_iid(id_interno_item)
    
    lista_sesgos_peliculas.append((id_pelicula_original, sesgo_valor))

df_sesgos_items = pd.DataFrame(lista_sesgos_peliculas, columns=['movieId', 'bi'])

# Unimos con los titulos antes de intentar acceder a 'title'
df_sesgos_items = df_sesgos_items.merge(peliculas[['movieId', 'title']], on='movieId', how='left')

print("Peliculas con sesgo mas alto:")
display(df_sesgos_items.sort_values(by='bi', ascending=False).head(5)[['movieId', 'title', 'bi']])

print("\nPeliculas con sesgo mas bajo:")
display(df_sesgos_items.sort_values(by='bi', ascending=True).head(5)[['movieId', 'title', 'bi']])

Peliculas con sesgo mas alto:


,movieId,title,bi
232,318,"Shawshank Redemption, The (1994)",0.915512
2395,1204,Lawrence of Arabia (1962),0.848283
722,750,Dr. Strangelove or: How I Learned to Stop Worr...,0.844607
332,904,Rear Window (1954),0.804869
2427,1104,"Streetcar Named Desire, A (1951)",0.803311



Peliculas con sesgo mas bajo:


,movieId,title,bi
1490,1556,Speed 2: Cruise Control (1997),-1.348457
3041,3593,Battlefield Earth (2000),-1.238729
4059,1760,Spice World (1997),-1.226927
1486,1499,Anaconda (1997),-1.200242
989,1882,Godzilla (1998),-1.186551


#### Sesgo usarios
Igual hemos hecho con el sesgo de peliculas, el sesgo de usuario, $b_u$, modela la tendencia individual del usuario a puntuar. En este punto podemos preciar que dependiendo de si es positivo o negativo
- Sesgo positivo: Representa a los usuarios que son generosos y que tienden a puntuar por encima de la media global
- Sesgo negativo: Representa a los usuarios estrictos, que tienden a puntuar por debajo de la media global.

Este componente es crucial para normalizar la matriz de utilidad y asegurar que las recomendaciones se basen en las preferencias relativas y no solo en la escala de puntuación personal del usuario

In [9]:
# Recoger sesgos usuarios
sesgos_bu = modelo_reentrenado.bu

lista_sesgos_usuarios = []
for id_interno_usuario in conjunto_entrenamiento_nuevo.all_users():
    # Obtener el sesgo usando el índice (ID interno)
    sesgo_valor = sesgos_bu[id_interno_usuario]
    
    # Mapear el ID interno al userId original (string)
    id_usuario_original = conjunto_entrenamiento_nuevo.to_raw_uid(id_interno_usuario)
    
    lista_sesgos_usuarios.append((id_usuario_original, sesgo_valor))


df_sesgos_usuarios = pd.DataFrame(lista_sesgos_usuarios, columns=['userId', 'bu'])

print("Usuarios con sesgo mas alto:")
display(df_sesgos_usuarios.sort_values(by='bu', ascending=False).head(5))

print("\nUsuarios con sesgo mas bajo):")
display(df_sesgos_usuarios.sort_values(by='bu', ascending=True).head(5))

Usuarios con sesgo mas alto:


,userId,bu
52,53,1.199041
42,43,1.126081
451,452,0.960937
275,276,0.907727
11,12,0.898600



Usuarios con sesgo mas bajo):


,userId,bu
441,442,-1.826163
152,153,-1.404871
138,139,-1.400646
566,567,-1.370398
507,508,-1.326106


#### Polularidad
Si bien el sesgo $b_i$ mide la calidad percibida de una pelicula, el conteo de ratings mide su actividad o popularidad historica. Esto nos ayuda a entender el sesgo hacia la popularidad de los filtros colaborativos.

In [10]:
conteo_ratings_peliculas = ratings_combinados.groupby('movieId').size().reset_index(name='conteo_ratings')

# Unimos con títulos
df_popularidad_peliculas = conteo_ratings_peliculas.merge(peliculas[['movieId', 'title']], on='movieId', how='left')

print("Peliculas mas populares:")
display(df_popularidad_peliculas.sort_values(by='conteo_ratings', ascending=False).head(5))

Peliculas mas populares:


,movieId,conteo_ratings,title
4369,356,329,Forrest Gump (1994)
3903,318,317,"Shawshank Redemption, The (1994)"
3640,296,307,Pulp Fiction (1994)
6822,593,279,"Silence of the Lambs, The (1991)"
2985,2571,278,"Matrix, The (1999)"


### Análisis en el espacio latente
La Factorizacion matricial de Bajo Rango (SVD) descompone la matriz de utilidad en dos matrices de factores latentes:
- $\mathbf{P}$ (para usuarios, $\mathbf{p}_u$) 
- $\mathbf{Q}$ (para ítems, $\mathbf{q}_i$)

Cada fila de $\mathbf{Q}$ (o $\mathbf{q}_i$) representa una pelicula en un espacio vectorial de $n=50$ dimensiones latentes. Nuestro objetivo sera encontrar las 10 peliculas más priximas a una pelicula dada dentro de este espacio. Para ello usaremos la disctancia Euclidea:

$$\text{Distancia}(i_1, i_2) = ||\mathbf{q}_{i_1} - \mathbf{q}_{i_2}||$$

Una menor distancia euclídea implica mayor similitud entre las películas, haciendonos ver que tienen patrones de consumo o preferencias latentes similares

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

matriz_factores_peliculas = modelo_reentrenado.qi
df_factores_peliculas = pd.DataFrame(matriz_factores_peliculas)

pelicula_referencia = "Toy Story (1995)"

# Obtenenemos su movieId
id_pelicula_original = peliculas[peliculas['title'] == pelicula_referencia]['movieId'].iloc[0]

#Obtenemos si ID interno
id_interno_referencia = conjunto_entrenamiento_nuevo.to_inner_iid(id_pelicula_original)

# Extraemos el vector de factores latentes de referencia
vector_referencia = df_factores_peliculas.iloc[id_interno_referencia].values


#### Calculo de distancia
Ahora que tenemos los datos que nos interesan de la pelicula concreta, vamos a calcular la distancia euclidea para despues quedarlos con los 10 que tengan la menor distancia a la pelicula de la cual buscamos

In [12]:
distancia = []

for id_interno_actual in conjunto_entrenamiento_nuevo.all_items():
    if id_interno_actual == id_interno_referencia:
        continue

    vector_actual = df_factores_peliculas.iloc[id_interno_actual].values
    distancia_valor = euclidean_distances(
        vector_referencia.reshape(1, -1), 
        vector_actual.reshape(1, -1)
    )[0][0]

    id_original = conjunto_entrenamiento_nuevo.to_raw_iid(id_interno_actual)
    distancia.append((id_original, distancia_valor))

# Buscamos las 10 con menos distancia
distancia.sort(key=lambda x: x[1], reverse=False)
diez_similares = distancia[:10]

#Creamos DataFrame
df_similitud = pd.DataFrame(diez_similares, columns=['movieId', 'Distancia_Euclidea'])

# Unimos con la información de la pelicula
df_similitud = df_similitud.merge(peliculas[['movieId', 'title', 'genres']], on='movieId', how='left')

# Mostramos las peliculas
display(df_similitud)

,movieId,Distancia_Euclidea,title,genres
0,8183,0.812496,Educating Rita (1983),Comedy|Drama
1,1120,0.812682,"People vs. Larry Flynt, The (1996)",Comedy|Drama
2,7088,0.813454,Black Orpheus (Orfeu Negro) (1959),Drama|Romance
3,56339,0.818169,"Orphanage, The (Orfanato, El) (2007)",Drama|Horror|Mystery|Thriller
4,77177,0.820473,Wild China (2008),Documentary
5,3251,0.821180,Agnes of God (1985),Drama|Mystery
6,114662,0.822549,American Sniper (2014),Action|War
7,94478,0.822990,Dark Shadows (2012),Comedy|Horror|IMAX
8,7319,0.824660,Club Dread (2004),Comedy|Horror
9,2473,0.826051,Soul Man (1986),Comedy


### Tecnica de clustering
Aprovechando que he hemos dado en Sistemas Inteligentes en la asignatura de tercero de carrera voy a utilizar el algoritmo K-means, que ademas queda bastante adecuado para esta tarea.

#### Preparacion

In [13]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

matriz_factores_usuarios = modelo_reentrenado.pu
df_factores_usuarios = pd.DataFrame(matriz_factores_usuarios)

# Creamos un escalador 
escalador = StandardScaler()
# Ajustamos y transformamos los datos
datos_escalados = escalador.fit_transform(df_factores_usuarios)

#### Aplicacion k-means
Este algoritmo funciona de la siguiente formaÑ
- Datos de Entrada: K-Means utiliza la matriz de factores latentes de usuarios, donde cada fila es un vector de 50 dimensiones que describe las preferencias implicitas de un usuario. Estos vectores son la representacion numérica de su perfil de gustos.
- Inicialización: Se elige un número $K$ de clusters y se inicializan $K$ puntos aleatorios llamados centroides.
- Asignación: Cada usuario es asignado al centroide más cercano en el espacio de 50 dimensiones, basándose en la distancia euclídea. Un usuario está más cerca de un centroide si su vector de preferencias es más parecido a la media de ese grupo.
- Actualización: Los centroides se recalculan como la media de todos los usuarios asignados a su cluster.
- Iteración: Los pasos se repiten hasta que los centroides dejan de moverse significativamente, lo que indica que se han formado grupos estables de usuarios similares.

In [14]:
kmeans = KMeans(
    n_clusters=5, 
    random_state=42, 
    n_init=10
)

kmeans.fit(datos_escalados)
# Obtenemos las etiquetas
etiquetas = kmeans.labels_

#Añadimos las etiquetas al DataFrame de factores de usuario
df_factores_usuarios['cluster'] = etiquetas

mapa_id_interno_usuario = {
    i: conjunto_entrenamiento_nuevo.to_raw_uid(i)
    for i in conjunto_entrenamiento_nuevo.all_users()
}

df_factores_usuarios['userId'] = df_factores_usuarios.index.map(mapa_id_interno_usuario)

#### Interpretacion y visualizacion
Para interpretar los segmentos, analizaremos la actividad promedio y el sesgo promedio ($b_u$) de los usuarios dentro de cada cluster.

In [15]:
# Preparamos los datos de actividad y sesgos
df_actividad_usuario = ratings_combinados.groupby('userId').size().reset_index(name='conteo_ratings')
df_sesgos_bu = df_sesgos_usuarios[['userId', 'bu']]

# Unimos los datos de actividad y sesgos con el DataFrame de clusters
df_segmentacion = df_factores_usuarios[['userId', 'cluster']].merge(
    df_actividad_usuario, on='userId', how='left'
)
df_segmentacion = df_segmentacion.merge(df_sesgos_bu, on='userId', how='left')

# Calculamos las métricas promedio por cluster
analisis_segmentos = df_segmentacion.groupby('cluster').agg(
    Tamano_Segmento=('userId', 'count'),
    Conteo_Ratings_Promedio=('conteo_ratings', 'mean'),
    Sesgo_Promedio=('bu', 'mean')
).reset_index()

display(analisis_segmentos)

,cluster,Tamano_Segmento,Conteo_Ratings_Promedio,Sesgo_Promedio
0,0,302,117.794702,0.053007
1,1,36,635.000000,-0.320430
2,2,1,437.000000,-0.694774
3,3,271,153.394834,-0.021569
4,4,1,400.000000,-1.294188


### Bibliografia
Por último, dejo algunos enlaces que me han servido de bastante ayuda para la programacion de determinadas cosas y para el entendimiento de otras.
- Wikipedia. (s. f.). K-means clustering. https://en.wikipedia.org/wiki/K-means_clustering
IEBS Business School. (s. f.). 
- Algoritmo K-means: qué es y cómo funciona. https://www.iebschool.com/hub/algoritmo-k-means-que-es-y-como-funciona-big-data/
- Surprise. (s. f.). Surprise: A Python scikit for building and analyzing recommender systems. https://surpriselib.com/
- De Rojas Parra. (s. f.). Explorando la biblioteca Surprise en Python para sistemas de recomendación [Artículo en LinkedIn]. https://www.linkedin.com/pulse/explorando-la-biblioteca-surprise-en-python-para-de-rojas-parra-8l2be/
- DataCamp. (s. f.). Euclidean distance. https://www.datacamp.com/es/tutorial/euclidean-distance
- Wikipedia. (s. f.). Singular value decomposition. https://en.wikipedia.org/wiki/Singular_value_decomposition
- Aazg24. (s. f.). Transformación de imágenes con SVD usando Python, NumPy y Matplotlib: cómo aplicar la descomposición en valores singulares [Entrada en Medium]. https://medium.com/@aazg24/transformación-de-imágenes-con-svd-usando-python-numpy-y-matplotlib-️-cómo-aplicar-la-9f38820fc020
- nteractiveChaos. (s. f.). Descomposición en valores singulares (SVD). https://interactivechaos.com/es/wiki/descomposicion-en-valores-singulares-svd